In [8]:
import numpy as np
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import folium
from folium.plugins import StripePattern, ScrollZoomToggler, DualMap, FloatImage
import branca.colormap as cm
import random
import json
import warnings
import shapely
import os

warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=UserWarning)

In [9]:
actual_results = pd.read_excel("../redistricting/2022_congressional_voting_results.xlsx")
actual_results["district"] = actual_results["district"].astype(str)
run = "moving_knife"
os.makedirs(os.path.join("folium_maps",run), exist_ok=True)

In [10]:
def dissolve_df(df, buffer_width=1e-4):
    dissolved = df.dissolve(by=f'DISTRICT_', aggfunc="sum").reset_index()
    dissolved = dissolved[['geometry', 'POP20', 'USH20_D', 'USH20_R']]
    dissolved['geometry'] = dissolved.buffer(buffer_width)
    dissolved['district'] = dissolved.index
    dissolved['district'] = dissolved['district'] + 1
    dissolved["Dem_PCT"] = round(
        dissolved['USH20_D']/(dissolved['USH20_D']+dissolved['USH20_R']), 4)*100
    return dissolved

In [11]:
# Voting Colorings

voting_colors = [
    (154, 4, 11, 1.0),
    (230, 101, 90, 1.0),
    (255, 211, 195, 1.0),
    (255, 227, 128, 1.0),
    (187, 230, 248, 1.0),
    (81, 148, 195, 1.0),
    (1, 68, 109, 1.0),
]
voting_colors = [(x[0]/255., x[1]/255., x[2]/255., x[3])
                 for x in voting_colors]


voting_cm = cm.StepColormap(
    colors=voting_colors,
    index=[0, 35, 45, 47.5, 52.5, 55, 65],
    vmin=0,
    vmax=100,
    tick_labels=[0, 35, 45, 50, 55, 65, 100],

)
voting_cm.caption = "Democratic Voting Percentage (%)"


map_styles = {
    "voting": {"colorscale": voting_cm, "value": "Dem_PCT"},
}


def polygon_styler(feature):
    value = feature['properties'][map_styles[map_type]["value"]]
    # print(map_styles[map_type]["colorscale"](value))
    return {"fillColor": map_styles[map_type]["colorscale"](value), "color": "black", "weight": 2, "opacity": 1, "fillOpacity": 0.60}

def voting_polygon_styler(feature):
    value = feature['properties'][map_styles["voting"]["value"]]
    # print(map_styles[map_type]["colorscale"](value))
    return {"fillColor": map_styles["voting"]["colorscale"](value), "color": "black", "weight": 2, "opacity": 1, "fillOpacity": 0.60}


def voting_marker_text(value):
    text = ""
    voting_bounds = [45, 47.5, 50, 52.5, 55]
    # voting_text = ["R", "R*", "R**", "D**", "D*"]
    voting_text = ["", "C", "C*", "C*", "C"]
    for b_i, b_v in enumerate(voting_bounds):
        if value <= b_v:
            text = voting_text[b_i]
            break
    return text


def district_marker_text(value):
    return str(int(value))


marker_styles = ['font-size: 18pt;',
                 'text-shadow: -2px 0 white, 0 2px white, 2px 0 white, 0 -2px white;']
marker_style = " ".join(marker_styles)


In [12]:
def prepare_state(state, actual_districts_path=None):
    ## Read Data
    algo_districts = gpd.read_file(
        f"../output/{state.lower()}_2020/{state.lower()}_2020_{run}.shp"
    )
    if actual_districts_path:
        actual_districts = gpd.read_file(actual_districts_path)
    else:
        actual_districts = gpd.read_file(
            f"../data/{state.lower()}/district-shapes/POLYGON.shp"
        )
    actual_districts["geometry"] = (
        actual_districts["geometry"].apply(shapely.validation.make_valid).buffer(1e-9)
    )
    # Needs Projected Coordinate System (Longitude, Latitude)
    actual_districts = actual_districts.to_crs("EPSG:4326")

    ## Merge Actual Districts Geographic Data with Results
    state_results = actual_results.loc[actual_results["state"] == state]
    actual_districts = actual_districts.merge(
        state_results, left_on="NAME", right_on="district"
    )
    actual_districts = actual_districts[
        ["geometry", "NAME", "republican", "democrat", "district"]
    ]
    actual_districts["Dem_PCT"] = (
        100
        * actual_districts["democrat"]
        / (actual_districts["democrat"] + actual_districts["republican"])
    )
    actual_districts["Dem_PCT"] = actual_districts["Dem_PCT"].apply(
        lambda x: round(x, 2)
    )
    actual_districts["district"] = actual_districts["district"].astype(int)

    ## Project to the algorithm redistricting results
    projected = algo_districts.to_crs("EPSG:4326")
    projected["geometry"] = (
        projected["geometry"].apply(shapely.validation.make_valid).buffer(1e-9)
    )
    projected = projected[["POP20", "USH20_D", "USH20_R", "geometry", "DISTRICT_"]]

    ## Get bounds of geographic area to be mapped.
    projected["REPRESENTATIVE_POINT"] = projected.centroid
    projected["RP_LON"] = projected["REPRESENTATIVE_POINT"].apply(lambda p: p.x)
    projected["RP_LAT"] = projected["REPRESENTATIVE_POINT"].apply(lambda p: p.y)

    sw_corner = projected[["RP_LAT", "RP_LON"]].min().values.tolist()
    ne_corner = projected[["RP_LAT", "RP_LON"]].max().values.tolist()

    ## Create world mask to hide areas outside of the state boundaries
    world_mask = shapely.geometry.Polygon(
        ((-180, -90), (180, -90), (180, 90), (-180, 90))
    )
    world_mask = gpd.GeoDataFrame(index=[0], geometry=[world_mask], crs="EPSG:4326")

    # Algorithm Mask
    projected_dissolved = projected.dissolve()
    projected_dissolved["geometry"] = projected_dissolved.buffer(1e-3)
    anti_clip_algo = gpd.overlay(world_mask, projected_dissolved, how="difference")

    # Actual Mask
    actual_dissolved = actual_districts.dissolve()
    anti_clip_actual = gpd.overlay(world_mask, actual_dissolved, how="difference")

    return {
        "corners": [sw_corner, ne_corner],
        "mask_actual": anti_clip_actual,
        "mask_algo": anti_clip_algo,
        "districts_actual": actual_districts,
        "districts_algo": projected,
    }


def create_mask(
    mask,
    style_fn=lambda x: {"fillColor": "white", "fillOpacity": "95%", "color": "black"},
):
    mask_group = folium.FeatureGroup(name="Mask", show=True)
    for _, r in mask.iterrows():
        sim_geo = gpd.GeoSeries(r["geometry"]).simplify(tolerance=0.001)
        geo_j = sim_geo.to_json()
        geo_j = folium.GeoJson(data=geo_j, style_function=style_fn)
        geo_j.add_to(mask_group)
    return mask_group


def create_voting_districts(districts, label=None, style_fn=voting_polygon_styler):
    district_group = folium.FeatureGroup(name=label, show=True)
    for _, r in districts.iterrows():
        sim_geo = gpd.GeoSeries(r["geometry"]).simplify(tolerance=0.001)
        text = voting_marker_text(r[map_styles["voting"]["value"]])
        # Polygon
        geo_j = json.loads(sim_geo.to_json())
        geo_j["features"][0]["properties"] = {
            "Dem_PCT": r["Dem_PCT"],
            "district": int(r["NAME"]),
        }
        geo_j = folium.GeoJson(data=geo_j, style_function=style_fn)
        folium.Popup(f'Dem%: {r["Dem_PCT"]}').add_to(geo_j)
        geo_j.add_to(district_group)

        # Text Marker @ Representative Point
        rp = sim_geo.centroid
        folium.map.Marker(
            location=[rp[0].y, rp[0].x],
            icon=folium.features.DivIcon(
                html=f'<div style="{marker_style}">{text}</div>'
            ),
        ).add_to(district_group)
    return district_group


def create_algo_voting_districts(districts, label=None, style_fn=voting_polygon_styler):
    district_group = folium.FeatureGroup(name=label)
    for _, r in districts.iterrows():
        sim_geo = gpd.GeoSeries(r["geometry"]).simplify(tolerance=0.001)
        text = voting_marker_text(r[map_styles["voting"]["value"]])
        # Polygon
        geo_j = json.loads(sim_geo.to_json())
        geo_j["features"][0]["properties"] = {
            "Dem_PCT": r["Dem_PCT"],
            "district": r["district"],
        }
        geo_j = folium.GeoJson(data=geo_j, style_function=style_fn)
        folium.Popup(f'Dem%: {r["Dem_PCT"]}').add_to(geo_j)
        geo_j.add_to(district_group)

        # Representative Point
        rp = sim_geo.centroid
        folium.map.Marker(
            location=[rp[0].y, rp[0].x],
            icon=folium.features.DivIcon(
                html=f'<div style="{marker_style}">{text}</div>'
            ),
        ).add_to(district_group)
    return district_group

def get_stats(state, algo_df):
    state_results = actual_results.loc[actual_results["state"] == state]
    n_districts = len(state_results)

    ## Party Vote
    actual_pv = (
        state_results[["democrat"]].sum()
        / state_results[["democrat", "republican"]].sum().sum()
    )
    actual_pv = round(actual_pv.to_list()[0] * 100, 1)

    algo_pv = algo_df[["USH20_D"]].sum() / algo_df[["USH20_D", "USH20_R"]].sum().sum()
    algo_pv = round(algo_pv.to_list()[0] * 100, 1)

    print(
        f"Party Vote (D, R): ({actual_pv}%, {100-actual_pv}%) | ({algo_pv}%, {100-algo_pv}%)"
    )

    ## Seats Won
    actual_seats = state_results.loc[
        state_results["democrat"] > state_results["republican"]
    ]
    actual_seats = len(actual_seats)

    algo_seats = algo_df.loc[algo_df["USH20_D"] > algo_df["USH20_R"]]
    algo_seats = len(algo_seats)
    print(
        f"Seats Won (D, R): ({actual_seats}, {n_districts-actual_seats}) | ({algo_seats}, {n_districts-algo_seats})"
    )

    ## Proportional Seating
    actual_prop_seating = round(actual_pv / 100.0 * n_districts)
    algo_prop_seating = round(algo_pv / 100.0 * n_districts)
    print(
        f"Proportional Seating (D, R): ({actual_prop_seating}, {n_districts-actual_prop_seating}) | ({algo_prop_seating}, {n_districts-algo_prop_seating})"
    )

    ## Competitiveness
    actual_non_comp = len(
        state_results.loc[
            (state_results["dem_share"] < 0.45) | (state_results["dem_share"] > 0.55)
        ]
    )
    actual_comp = len(
        state_results.loc[
            (
                (state_results["dem_share"] >= 0.45)
                & (state_results["dem_share"] < 0.475)
            )
            | (
                (state_results["dem_share"] >= 0.525)
                & (state_results["dem_share"] < 0.55)
            )
        ]
    )
    actual_very_comp = len(
        state_results.loc[
            (state_results["dem_share"] >= 0.475) & (state_results["dem_share"] < 0.525)
        ]
    )

    algo_non_comp = len(
        algo_df.loc[(algo_df["Dem_PCT"] < 45) | (algo_df["Dem_PCT"] > 55)]
    )
    algo_comp = len(
        algo_df.loc[
            ((algo_df["Dem_PCT"] >= 45) & (algo_df["Dem_PCT"] < 47.5))
            | ((algo_df["Dem_PCT"] >= 52.5) & (algo_df["Dem_PCT"] < 55))
        ]
    )
    algo_very_comp = len(
        algo_df.loc[(algo_df["Dem_PCT"] >= 47.5) & (algo_df["Dem_PCT"] < 52.5)]
    )

    print(
        f"Competitiveness (NC, C, C*): ({actual_non_comp}, {actual_comp}, {actual_very_comp}) | ({algo_non_comp}, {algo_comp}, {algo_very_comp})"
    )

In [13]:
map_tiles = 'https://stamen-tiles-{s}.a.ssl.fastly.net/toner-lite/{z}/{x}/{y}{r}.png'
map_attr = 'Map tiles by <a href="http://stamen.com">Stamen Design</a>, <a href="http://creativecommons.org/licenses/by/3.0">CC BY 3.0</a> &mdash; Map data &copy; <a href="https://www.openstreetmap.org/copyright">OpenStreetMap</a> contributors'

In [38]:
state_dict = dict(
    az_2020={"districts": 9, "crs": "EPSG:2223"},
    ca_2020={"districts": 52, "crs": "EPSG:2225"},
    ct_2020={"districts": 5, "crs": "EPSG:2234"},
    fl_2020={"districts": 28, "crs": "EPSG:2225"},
    ga_2020={"districts": 14, "crs": "EPSG:2240"},
    ma_2020={"districts": 9, "crs": "EPSG:2249"},
    md_2020={"districts": 8, "crs": "EPSG:2248"},
    mi_2020={"districts": 13, "crs": "EPSG:2252"},
    mn_2020={"districts": 8, "crs": "EPSG:2811"},
    nc_2020={"districts": 14, "crs": "EPSG:2264"},
    nh_2020={"districts": 2, "crs": "EPSG:3437"},
    nj_2020={"districts": 12, "crs": "EPSG:2824"},
    nv_2020={"districts": 4, "crs": "EPSG:2821"},
    oh_2020={"districts": 15, "crs": "EPSG:2834"},
    or_2020={"districts": 6, "crs": "EPSG:2269"},
    pa_2020={"districts": 17, "crs": "EPSG:2271"},
    sc_2020={"districts": 7, "crs": "EPSG:2273"},
    tx_2020={"districts": 38, "crs": "EPSG:2277"},
    va_2020={"districts": 11, "crs": "EPSG:2283"},
    wi_2020={"districts": 8, "crs": "EPSG:2288"},
)

In [39]:
run_name = "moving_knife"
output_dir = os.path.join("folium_maps", f"{run_name}")
os.makedirs(output_dir, exist_ok=True)

for state in state_dict:
    state = state[:2].upper()
    state_objects = prepare_state(state)

    m = DualMap(
        zoom_control=False,
        layout="horizontal",
        tiles=map_tiles,
        attr=map_attr,
        control_scale=True)
    m.fit_bounds(state_objects["corners"])

    # Map 1
    mask_actual_grp = create_mask(state_objects["mask_actual"])
    mask_actual_grp.add_to(m.m1)

    def ca_voting_polygon_styler(feature):
        value = feature['properties'][map_styles["voting"]["value"]]
        # print(map_styles[map_type]["colorscale"](value))
        return {"fillColor": map_styles["voting"]["colorscale"](value), "color": "black", "weight": 1, "opacity": 1, "fillOpacity": 0.60}

    actual_districts_group = create_voting_districts(state_objects["districts_actual"],style_fn=ca_voting_polygon_styler)
    actual_districts_group.add_to(m.m1)

    # Map 2
    mask_actual_grp = create_mask(state_objects["mask_algo"])
    mask_actual_grp.add_to(m.m2)

    projected_districts = dissolve_df(state_objects["districts_algo"])
    algo_districts_group = create_algo_voting_districts(projected_districts, style_fn=ca_voting_polygon_styler)
    algo_districts_group.add_to(m.m2)

    m.save(os.path.join("folium_maps", run, f"{run}_{state}.html"))
    get_stats(state,projected_districts)


Party Vote (D, R): (43.1%, 56.9%) | (49.9%, 50.1%)
Seats Won (D, R): (3, 6) | (5, 4)
Proportional Seating (D, R): (4, 5) | (4, 5)
Competitiveness (NC, C, C*): (6, 1, 2) | (7, 2, 0)
Party Vote (D, R): (63.6%, 36.4%) | (66.3%, 33.7%)
Seats Won (D, R): (40, 12) | (43, 9)
Proportional Seating (D, R): (33, 19) | (34, 18)
Competitiveness (NC, C, C*): (41, 6, 5) | (42, 4, 6)
Party Vote (D, R): (57.8%, 42.2%) | (60.2%, 39.8%)
Seats Won (D, R): (5, 0) | (5, 0)
Proportional Seating (D, R): (3, 2) | (3, 2)
Competitiveness (NC, C, C*): (4, 0, 1) | (4, 1, 0)
Party Vote (D, R): (40.5%, 59.5%) | (47.5%, 52.5%)
Seats Won (D, R): (8, 20) | (10, 18)
Proportional Seating (D, R): (11, 17) | (13, 15)
Competitiveness (NC, C, C*): (25, 2, 1) | (21, 5, 2)
Party Vote (D, R): (47.7%, 52.3%) | (49.0%, 51.0%)
Seats Won (D, R): (5, 9) | (5, 9)
Proportional Seating (D, R): (7, 7) | (7, 7)
Competitiveness (NC, C, C*): (13, 1, 0) | (11, 2, 1)
Party Vote (D, R): (69.8%, 30.200000000000003%) | (78.0%, 22.0%)
Seats Won 